In [1]:
import zipfile

In [4]:
with zipfile.ZipFile("../data/zippedData/im.db.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/zippedData/")

In [6]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("../data/zippedData/im.db")
cur = conn.cursor()

In [4]:
pd.read_sql(
'''
SELECT *
FROM sqlite_master
''', 
conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [5]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    movie_basics
ORDER BY
    start_year ASC
''', 
conn)

In [6]:
q1.head(15) # movie basics is a table showing all movies from 2010- and gives runtime and genres

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,Drama
1,tt0154039,So Much for Justice!,Oda az igazság,2010,100.0,History
2,tt0162942,Children of the Green Dragon,A zöld sárkány gyermekei,2010,89.0,Drama
3,tt0230212,The Final Journey,The Final Journey,2010,120.0,Drama
4,tt0312305,Quantum Quest: A Cassini Space Odyssey,Quantum Quest: A Cassini Space Odyssey,2010,45.0,"Adventure,Animation,Sci-Fi"
5,tt0326592,The Overnight,The Overnight,2010,88.0,None
6,tt0326965,In My Sleep,In My Sleep,2010,104.0,"Drama,Mystery,Thriller"
7,tt0331312,This Wretched Life,This Wretched Life,2010,99.0,"Comedy,Drama"
8,tt0337882,Blind Sided,Blind Sided,2010,NaN,"Comedy,Crime,Drama"
9,tt0378546,The Payback Man,The Payback Man,2010,NaN,"Crime,Drama,Thriller"


In [7]:
q1 = pd.read_sql(
'''
SELECT 
    principals.*,
    per.primary_name
FROM 
    principals
    INNER JOIN persons AS per ON principals.person_id = per.person_id
WHERE
    per.primary_name == "Hans Zimmer" 
   
''', 
conn)

q1

,movie_id,ordering,person_id,category,job,characters,primary_name
0,tt0816692,9,nm0001877,composer,None,None,Hans Zimmer
1,tt1210819,10,nm0001877,composer,None,None,Hans Zimmer
2,tt1270766,10,nm0001877,composer,None,None,Hans Zimmer
3,tt1341188,9,nm0001877,composer,None,None,Hans Zimmer
4,tt1375666,7,nm0001877,composer,None,None,Hans Zimmer
5,tt1531683,3,nm0001877,composer,None,None,Hans Zimmer
6,tt1578275,9,nm0001877,composer,None,None,Hans Zimmer
7,tt1823672,8,nm0001877,composer,None,None,Hans Zimmer
8,tt2514298,10,nm0001877,composer,None,None,Hans Zimmer
9,tt3062096,9,nm0001877,composer,None,None,Hans Zimmer


In [7]:
# Decided to also filter 'composer' due to it 'composer' being the main composer of the movie as 
# opposed to 'head composer', 'main composer', 'lead composer', etc. 
movie_info = pd.read_sql(
'''
SELECT 
    mb.movie_id,
    mb.primary_title,
    mb.original_title,
    mb.start_year,
    mb.genres,
    mr.averagerating AS average_rating,
    mr.numvotes AS num_votes,
    per.primary_name AS persons_name,
    princ.category AS persons_job
FROM 
    movie_basics AS mb
    INNER JOIN movie_ratings AS mr ON mb.movie_id = mr.movie_id
    INNER JOIN principals AS princ ON mb.movie_id = princ.movie_id
    INNER JOIN persons AS per ON princ.person_id = per.person_id
    INNER JOIN movie_akas AS ma ON mb.movie_id = ma.movie_id
WHERE
    princ.category IN ('actor', 'actress', 'director', 'writer', 'producer', 'composer', 'director of photography')
    AND ma.region == 'US'
ORDER BY
    mb.start_year ASC
''', 
conn)

In [8]:
movie_info

,movie_id,primary_title,original_title,start_year,genres,average_rating,num_votes,persons_name,persons_job
0,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Johnny Depp,actor
1,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Helena Bonham Carter,actress
2,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Tim Burton,director
3,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Anne Hathaway,actress
4,tt1014759,Alice in Wonderland,Alice in Wonderland,2010,"Adventure,Family,Fantasy",6.5,358813,Joe Roth,producer
...,...,...,...,...,...,...,...,...,...
214270,tt9562694,Alien Warfare,Alien Warfare,2019,"Action,Sci-Fi",2.6,1509,Scott C. Roe,actor
214271,tt9562694,Alien Warfare,Alien Warfare,2019,"Action,Sci-Fi",2.6,1509,David Meadows,actor
214272,tt9562694,Alien Warfare,Alien Warfare,2019,"Action,Sci-Fi",2.6,1509,Nathan Zoebl,writer
214273,tt9562694,Alien Warfare,Alien Warfare,2019,"Action,Sci-Fi",2.6,1509,Ben Bailey,writer


In [9]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    directors
''', 
conn)

In [10]:
q1.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [11]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    known_for
''', 
conn)

In [12]:
q1.head(10)

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534
5,nm0061865,tt6791238
6,nm0061865,tt0287072
7,nm0061865,tt1682940
8,nm0062070,tt1470654
9,nm0062070,tt0363631


In [13]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    movie_akas

''', 
conn)

In [14]:
q1

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [15]:
q1 = pd.read_sql(
'''
SELECT 
    DISTINCT region
FROM 
    movie_akas
WHERE
    region LIKE '%US%'
ORDER BY
    region DESC
''', 
conn)

In [16]:
q1

,region
0,US


In [17]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    movie_ratings
''', 
conn)

q1

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [18]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    persons
''', 
conn)

q1

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


In [19]:
q1 = pd.read_sql(
'''
SELECT 
    *
FROM 
    writers
''', 
conn)

q1

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087
...,...,...
255868,tt8999892,nm10122246
255869,tt8999974,nm10122357
255870,tt9001390,nm6711477
255871,tt9004986,nm4993825
